In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
!pip install geopy
import geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e1/58/9289c6a03116025cdb61461d99b2493daa4967a80b13755463d71a0affeb/geopy-2.4.0-py3-none-any.whl.metadata
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Using cached geopy-2.4.0-py3-none-any.whl (125 kB)


### Below are Functions for cleaning up strings and converting datatypes:

In [2]:
remove_chars_Assesement = ['$', ',', "\xa0"]
remove_chars_Taxes = ['$', ',', "\xa0", ' **']
remove_chars_Year = ['\xa0', ' *']
remove_chars_Address = ['\xa0']
#input characters or character patterns you want removed in the above format

def Year_Cleaner(df, col, chars):
    result = list()
    for i in df[col]:
        for char in chars:
            i = i.replace(char, '')
        result.append(i)
    return pd.Series(result).astype(int)

def Annual_Assesement_Cleaner(df, col, chars):
    result = list()
    for i in df[col]:
        for char in chars:
            i = i.replace(char, '')
        result.append(i)
    return pd.Series(result).astype(int).astype(float)

def Taxes_and_Fees_Cleaner(df, col, chars):
    result = list()
    for i in df[col]:
        i = i.replace('Taxes and fees will be published after Council adopts the budget in May. ', '0')
        for char in chars:
            i = i.replace(char, '')
        result.append(i)
    result_series = pd.Series(result).replace('0', np.nan).astype(float)
    return result_series

def Address_Obj_to_string(df, col, chars):
    result = list()
    for i in df[col]:
        for char in chars:
            i = i.replace(char, '')
        result.append(i)
    return pd.Series(result)


In [3]:
#df1 = pd.DataFrame(pd.read_csv('Land_Building_Table.csv')).drop(columns='Unnamed: 0').rename(columns={'A_Assesement' : 'year'})
#df1.set_index(['Num', 'year'], inplace=True)
#df1

#### The Cell Below cleans df2 and df3 by:
    - Dropping extraneous columns
    - Changing column names to be more descriptive
    - Converting dtypes from objects to floats and strings where appropriate
    - Creating a tax rates column

In [4]:
df2 = pd.DataFrame(pd.read_csv('New_table.csv')).drop(columns='Unnamed: 0').rename(columns={'A_Levy_Year' : 'year'})
df2['year'] = Year_Cleaner(df2, 'year', chars=remove_chars_Year)
df2['A_Annual_Assesement'] = Annual_Assesement_Cleaner(df2, 'A_Annual_Assesement', chars=remove_chars_Assesement)
df2['A_Taxes_and_Fees'] = Taxes_and_Fees_Cleaner(df2, 'A_Taxes_and_Fees', chars=remove_chars_Taxes)
df2.drop(df2.loc[df2['A_Annual_Assesement'] == 0].index, inplace=True)
df2.drop(df2.loc[df2['A_Taxes_and_Fees'] == 0].index, inplace=True)
df2.set_index(['Num', 'year'], inplace=True)
print(df2)

df3 = pd.DataFrame(pd.read_csv('Table.csv')).drop(columns='Unnamed: 0').rename(columns={'A_Account_Num':'Num'})
df3['A_Properties'] = Address_Obj_to_string(df3, 'A_Properties', chars=remove_chars_Address)

               A_Annual_Assesement  A_Taxes_and_Fees
Num      year                                       
50299400 2023            4395607.0           2410.80
         2022            4395607.0           2296.00
         2021            4395607.0           1960.00
         2020            4395607.0           1120.00
         2019            4395607.0           1120.00
...                            ...               ...
50469430 2015              56650.0            590.86
         2014              56650.0            590.86
         2013              56650.0            588.03
         2012              56650.0            565.37
         2011              56650.0            565.37

[484577 rows x 2 columns]


In [5]:
Oji_House = df3.iloc[3517]
Oji_House_Num = Oji_House[1]
Schuerhoff_House = df3.iloc[3519]
Schuerhoff_House_Num = Schuerhoff_House[1]

C:\Users\nschu\AppData\Local\Temp\ipykernel_30084\1454120206.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Oji_House_Num = Oji_House[1]
C:\Users\nschu\AppData\Local\Temp\ipykernel_30084\1454120206.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Schuerhoff_House_Num = Schuerhoff_House[1]


In [6]:
#df3.to_csv('Alexandria_Tax_Info.csv')
#df2.to_csv('Alexandria_Property_Holders.csv')
df3

,A_Properties,Num,A_SQFT
0,3000 S 28TH ST,50299410,1
1,3102 S 28TH ST,50299400,1
2,"3200 S 28TH ST, UNIT 101",50466800,1090
3,"3200 S 28TH ST, UNIT 102",50466810,1280
4,"3200 S 28TH ST, UNIT 201",50466820,700
...,...,...,...
39456,400 YOAKUM PY,50469430,0
39457,450 YOAKUM PY,50469440,1
39458,3630 ZABRISKIE DR,26312010,1
39459,3634 ZABRISKIE DR,26312020,1


## Street Address Decoding

In [32]:
from geopy.geocoders import Nominatim, GoogleV3
from geopy.exc import GeocoderTimedOut
import time

def geocode(address):
    try:
        geolocator = Nominatim(user_agent="sawyer_application")
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else (None, None)
    except GeocoderTimedOut:
        return geocode(address)

In [33]:
import pandas as pd
data = pd.read_csv("data/Alexandria_Tax_Info.csv")

In [34]:
addresses = data['A_Properties'].unique()
len(addresses)

38896

In [ ]:
# addresses = [
#     "1600 Amphitheatre Parkway, Mountain View, CA",
#     "221B Baker Street, London",
#     # Add more addresses here
# ]

coordinates = []

for address in addresses:
    # if "UNIT" in address:
    #     continue
    print(address)
    lat, long = geocode(address)
    if lat and long:
        coordinates.append((address, lat, long))
    else:
        coordinates.append((address, "Not found"))
    time.sleep(2)  # To prevent rate limiting

for item in coordinates:
    print(item)

In [36]:
addresses

array(['3000 S 28TH ST', '3102 S 28TH ST', '3200 S 28TH ST, UNIT 101',
       ..., '3630  ZABRISKIE DR', '3634  ZABRISKIE DR',
       '3638  ZABRISKIE DR'], dtype=object)